In [ ]:
# Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !git clone https://github.com/luost26/diffusion-point-cloud.git
# %cd diffusion-point-cloud/

In [ ]:
%cd /content/drive/MyDrive/diffusion-point-cloud

/content/drive/MyDrive/diffusion-point-cloud


In [ ]:
"""
PyTorch	≥ 1.6.0
h5py	not specified (we used 4.61.1)
tqdm	not specified
tensorboard	not specified (we used 2.5.0)
numpy	not specified (we used 1.20.2)
scipy	not specified (we used 1.6.2)
scikit-learn
"""

!pip3 install torch torchvision
!pip3 install h5py tqdm tensorboard numpy scipy scikit-learn plotly

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [ ]:
# !python3 test_gen.py --ckpt ./pretrained/GEN_airplane.pt --categories airplane

# Test an auto-encoder
# !python3 test_ae.py --ckpt ./pretrained/AE_all.pt --categories all


In [ ]:
# !python3 test_ae.py --ckpt ./pretrained/AE_chair.pt --categories chair

### Generating Out-Of-Distribution Datasets

### 1. Imports

In [ ]:
import os
import time
import argparse
import plotly.graph_objects as go
import torch
from tqdm.auto import tqdm

from utils.dataset import *
from utils.misc import *
from utils.data import *
from models.autoencoder import *
from evaluation import EMD_CD

In [ ]:
device = 'cuda'

# Checkpoint
ckpt = torch.load('./pretrained/AE_chair.pt')
seed_all(2020)

# Load datasets
train_dset = ShapeNetCore(
    path='./data/shapenet.hdf5',
    cates=['chair'],
    split='train',
    scale_mode='shape_unit'
)
train_loader = DataLoader(train_dset, batch_size=256, num_workers=0)

## 3. Utility Functions

In [ ]:
def plot_point_cloud(np_data, color='blue'):
  fig = go.Figure(
    data=[
        go.Scatter3d(
            x=np_data[:,0], y=np_data[:,1], z=np_data[:,2],
            mode='markers',
            marker=dict(size=1, color=color)
        )
    ],
    layout=dict(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
    )
  )
  fig.show()


In [ ]:
def sample_ood_points(boundary_points, p=10, variance=1e-3):
    """
    Params:
      boundary_points
        - (M x 256), where M is the number of boundary points
      variance
        - variance of Gaussian distribution around boundary point

    Returns:
      latent out-of-distribution points ((MP) x 256)
    """
    dist = torch.distributions.multivariate_normal.MultivariateNormal(
        loc=torch.zeros(256), covariance_matrix=variance*torch.eye(256)
    )
    num_samples = boundary_points.size(0)*p
    noise = dist.sample([num_samples]).to(device)
    repeated_boundary_points = boundary_points.repeat_interleave(p, dim=0)
    return repeated_boundary_points + noise

In [ ]:
def pairwise_cosine_similarity(batch_vectors):
  return torch.nn.functional.cosine_similarity(
      batch_vectors.unsqueeze(1), batch_vectors.unsqueeze(0), dim=-1
  )
# Example usage
batch_size = 3
vector_dim = 3
batch_vectors = torch.randn(batch_size, vector_dim)  # Example batch of vectors
print(batch_vectors)
similarity_matrix = pairwise_cosine_similarity(batch_vectors)
print(similarity_matrix)

print(torch.nn.functional.cosine_similarity(batch_vectors[0:1, :], batch_vectors[1:2, :], dim=1))

tensor([[ 1.2372, -0.9604,  1.5415],
        [-0.4079,  0.8806,  0.0529],
        [ 0.0751,  0.4777, -0.6759]])
tensor([[ 1.0000, -0.5940, -0.7708],
        [-0.5940,  1.0000,  0.4385],
        [-0.7708,  0.4385,  1.0000]])
tensor([-0.5940])


In [ ]:
def find_latent_ood_points(points, k=100, m=50):
    """
    Params:
      points
        - (N x 256), where N is the number of points
      k
        - the k in k-NN distance
      m
        - the number of boundary points

    Returns:
      A tuple with two items
         1. latent ood points
         2. indices in `points` of the corresponding boundary
       point for each ood point
    """
    def get_kth_dists(pts1, pts2):
      # what we want: dists = get_dist(pts1, pts2)
      #print(pts1.shape)
      #print(pts2.shape)
      diffs = pts1.unsqueeze(1)-pts2.unsqueeze(0)
      print(diffs.shape)
      dists = torch.sqrt(torch.sum(diffs ** 2, dim=2))
      sorted_dists, _ = torch.sort(dists, dim=1)
      return torch.sort(sorted_dists[:, k], dim=0, descending=True, stable=True)

    def min_row_variance(tensor, dim=1):
      row_means = torch.mean(tensor, dim=dim, keepdim=True)
      squared_diffs = (tensor - row_means)**2
      row_variances = torch.mean(squared_diffs, dim=dim)
      return torch.min(row_variances).item()

    # 1. find boundary points
    kth_dists, indices = get_kth_dists(points, points)
    boundary_points = points[indices[:m]]

    # 2. sample ood points from each boundary point according to a
    # Gaussian distribution centered around the boundary point.
    p = 10
    ood_points = sample_ood_points(boundary_points, p=p, variance=1e-6)
    ood_kth_dists, ood_indices = get_kth_dists(ood_points, points)
    index_map = indices[:m].repeat_interleave(p, dim=0)

    # filter out generated ood points whose kth nearest neighbor is not
    # as far away as the largest kth nearest neighbor distance in the original
    # set of points.
    threshold = kth_dists[0].item()
    mask = torch.gt(ood_kth_dists, threshold)
    filtered_indices = ood_indices[mask.squeeze()]
    return ood_points[filtered_indices], index_map[filtered_indices]

## 4. Model

In [ ]:
model = AutoEncoder(ckpt['args']).to(device)
model.load_state_dict(ckpt['state_dict'])

<All keys matched successfully>

In [ ]:
all_oods = []
refs_for_oods = []
ood_ref_indices = []
cates_for_oods = []

for i, batch in enumerate(tqdm(train_loader)):
    print(f"Processing batch {i}...")
    ref = batch['pointcloud'].to(device)
    shift = batch['shift'].to(device)
    scale = batch['scale'].to(device)
    cate = batch['cate']
    model.eval()
    with torch.no_grad():
      code = model.encode(ref)
      latent_ood_points, ref_indices = find_latent_ood_points(code.detach())
      ood_points = model.decode(latent_ood_points, ref.size(1)).detach()

      ref = ref[ref_indices] * scale[ref_indices] + shift[ref_indices]
      ood_points = ood_points * scale[ref_indices] + shift[ref_indices]
      refs_for_oods.append(ref.squeeze().detach())
      all_oods.append(ood_points.detach())
      ood_ref_indices.append(ref_indices)
      filtered_cate = [cate[i] for i in ref_indices]
    cates_for_oods.extend(filtered_cate)

refs_for_oods = torch.cat(refs_for_oods, dim=0)
all_oods = torch.cat(all_oods, dim=0)

  0%|          | 0/22 [00:00<?, ?it/s]

Processing batch 0...
torch.Size([256, 256, 256])
tensor([[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 7.0274e-02, -7.7085e-02, -9.3813e-02,  ...,  2.1851e-03,
          -6.2737e-03,  6.2459e-02],
         [ 1.1257e-01, -5.4547e-02, -1.7927e-01,  ...,  3.3924e-02,
           8.0913e-02,  1.0600e-01],
         ...,
         [ 2.8915e-02, -1.0883e-01, -8.4751e-02,  ...,  2.8087e-02,
          -1.2669e-02,  4.8519e-02],
         [ 4.4112e-03, -8.5510e-02, -1.4021e-01,  ...,  2.3483e-02,
          -5.3905e-02,  2.9860e-02],
         [ 3.6779e-02, -1.0084e-01, -9.6373e-02,  ...,  3.3838e-02,
          -1.1829e-01,  4.5088e-02]],

        [[-7.0274e-02,  7.7085e-02,  9.3813e-02,  ..., -2.1851e-03,
           6.2737e-03, -6.2459e-02],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 4.2296e-02,  2.2538e-02, -8.5454e-02,  ...,  3.1739e-02,
           8.7187e-02,  4.

KeyboardInterrupt: 

In [ ]:
ood_synsetid_to_pc = {}
for i in range(len(cates_for_oods)):
  pc = all_oods[i]
  synsetid = cate_to_synsetid[cates_for_oods[i]]
  if synsetid not in ood_synsetid_to_pc:
    ood_synsetid_to_pc[synsetid] = []
  ood_synsetid_to_pc[synsetid].append(torch.unsqueeze(pc, 0))

for key in ood_synsetid_to_pc:
  ood_synsetid_to_pc[key] = torch.concat(ood_synsetid_to_pc[key])
  print(key)
  print(ood_synsetid_to_pc[key].shape)

03001627
torch.Size([107, 2048, 3])


In [ ]:
i = torch.randint(low=0, high=all_oods.shape[0]-1, size=(1,1)).item()
pc_truth = refs_for_oods[i].cpu().numpy()
pc_ood = all_oods[i].cpu().numpy()
print(all_oods.shape)
plot_point_cloud(pc_truth, color='blue')
plot_point_cloud(pc_ood, color='red')

torch.Size([107, 2048, 3])


### Outlier Dataset Generation






In [ ]:
import h5py
import numpy as np

In [ ]:
new_dataset_path = "./data/shapenet_augmented_chair.hdf5"

In [ ]:
original=h5py.File("./data/shapenet.hdf5", 'r')
with h5py.File(new_dataset_path, 'a') as f:
   # Append new data to it
   for key in original.keys():
      group = f.require_group(key)
      old_data = original[key]['train']
      print("old data shape:", old_data.shape)
      if key in ood_synsetid_to_pc:
        new_data = ood_synsetid_to_pc[key].cpu().numpy()
        print(new_data.shape)
        combined_data = np.concatenate([old_data, new_data], axis=0)
      else:
        combined_data = np.copy(old_data)
      print("new data shape:", combined_data.shape)
      try:
        group.create_dataset(f'train', data=combined_data)
      except:
        pass

      try:
        group.create_dataset(f'test', data=original[key]['test'])
        group.create_dataset(f'val', data=original[key]['val'])
      except:
        pass
original.close()

old data shape: (3438, 2048, 3)
new data shape: (3438, 2048, 3)
old data shape: (273, 2048, 3)
new data shape: (273, 2048, 3)
old data shape: (70, 2048, 3)
new data shape: (70, 2048, 3)
old data shape: (94, 2048, 3)
new data shape: (94, 2048, 3)
old data shape: (727, 2048, 3)
new data shape: (727, 2048, 3)
old data shape: (198, 2048, 3)
new data shape: (198, 2048, 3)
old data shape: (1536, 2048, 3)
new data shape: (1536, 2048, 3)
old data shape: (62, 2048, 3)
new data shape: (62, 2048, 3)
old data shape: (383, 2048, 3)
new data shape: (383, 2048, 3)
old data shape: (407, 2048, 3)
new data shape: (407, 2048, 3)
old data shape: (149, 2048, 3)
new data shape: (149, 2048, 3)
old data shape: (771, 2048, 3)
new data shape: (771, 2048, 3)
old data shape: (1311, 2048, 3)
new data shape: (1311, 2048, 3)
old data shape: (96, 2048, 3)
new data shape: (96, 2048, 3)
old data shape: (91, 2048, 3)
new data shape: (91, 2048, 3)
old data shape: (47, 2048, 3)
new data shape: (47, 2048, 3)
old data shape

In [ ]:
with h5py.File(new_dataset_path, 'a') as f:
  for key in f.keys():
    print(f[key]['val'].shape)

(607, 2048, 3)
(49, 2048, 3)
(13, 2048, 3)
(17, 2048, 3)
(129, 2048, 3)
(35, 2048, 3)
(272, 2048, 3)
(11, 2048, 3)
(68, 2048, 3)
(72, 2048, 3)
(27, 2048, 3)
(137, 2048, 3)
(232, 2048, 3)
(17, 2048, 3)
(17, 2048, 3)
(9, 2048, 3)
(528, 2048, 3)
(74, 2048, 3)
(989, 2048, 3)
(98, 2048, 3)
(10, 2048, 3)
(14, 2048, 3)
(164, 2048, 3)
(11, 2048, 3)
(112, 2048, 3)
(45, 2048, 3)
(120, 2048, 3)
(25, 2048, 3)
(85, 2048, 3)
(64, 2048, 3)
(348, 2048, 3)
(68, 2048, 3)
(240, 2048, 3)
(15, 2048, 3)
(11, 2048, 3)
(23, 2048, 3)
(51, 2048, 3)
(33, 2048, 3)
(36, 2048, 3)
(15, 2048, 3)
(40, 2048, 3)
(91, 2048, 3)
(25, 2048, 3)
(10, 2048, 3)
(356, 2048, 3)
(13, 2048, 3)
(23, 2048, 3)
(474, 2048, 3)
(33, 2048, 3)
(1258, 2048, 3)
(85, 2048, 3)
(19, 2048, 3)
(59, 2048, 3)
(291, 2048, 3)
(26, 2048, 3)


### Comparison of Models between Inlier and Outlier Datasets

---



In [ ]:
augmented_categories = [synsetid_to_cate[id] for id in ood_synsetid_to_pc]

In [ ]:
augmented_categories

['chair']

In [ ]:
print(','.join(augmented_categories))

chair,table,car,sofa,pot,microwave,bathtub,can,speaker,lamp,printer,file,bed,cabinet,vessel,bus,washer,dishwasher,clock,bowl,tin_can,jar,camera


In [ ]:
# train model on augmented dataset
!python3 train_ae.py --dataset_path {new_dataset_path} --log_root "./logs_ae_chair" --categories "chair"


Streaming output truncated to the last 5000 lines.
[2024-04-10 13:02:55,673::train::INFO] [Train] Iter 15079 | Loss 0.277898 | Grad 0.0433 
[2024-04-10 13:02:55,792::train::INFO] [Train] Iter 15080 | Loss 0.335221 | Grad 0.0518 
[2024-04-10 13:02:55,915::train::INFO] [Train] Iter 15081 | Loss 0.339682 | Grad 0.0536 
[2024-04-10 13:02:56,033::train::INFO] [Train] Iter 15082 | Loss 0.321753 | Grad 0.0579 
[2024-04-10 13:02:56,152::train::INFO] [Train] Iter 15083 | Loss 0.329795 | Grad 0.0664 
[2024-04-10 13:02:56,271::train::INFO] [Train] Iter 15084 | Loss 0.326985 | Grad 0.0413 
[2024-04-10 13:02:56,389::train::INFO] [Train] Iter 15085 | Loss 0.276319 | Grad 0.0452 
[2024-04-10 13:02:56,509::train::INFO] [Train] Iter 15086 | Loss 0.314650 | Grad 0.0651 
[2024-04-10 13:02:56,628::train::INFO] [Train] Iter 15087 | Loss 0.358193 | Grad 0.0649 
[2024-04-10 13:02:56,747::train::INFO] [Train] Iter 15088 | Loss 0.294484 | Grad 0.0833 
[2024-04-10 13:02:56,866::train::INFO] [Train] Iter 15089 |

In [ ]:
%mv './logs_ae_chair/AE_2024_04_10__12_19_39/ckpt_0.001665_1000.pt' './pretrained/AE_augmented_chair.pt'

In [ ]:
# test model
!python3 test_ae.py --ckpt './pretrained/AE_augmented_chair.pt' --categories chair


[2024-04-05 16:23:30,588::test::INFO] [ARGS::ckpt] './pretrained/AE_augmented_cosine_similarity.pt'
[2024-04-05 16:23:30,589::test::INFO] [ARGS::categories] ['chair', 'table', 'car', 'sofa', 'pot', 'microwave', 'bathtub', 'can', 'speaker', 'lamp', 'printer', 'file', 'bed', 'cabinet', 'vessel', 'bus', 'washer', 'dishwasher', 'clock', 'bowl', 'tin_can', 'jar', 'camera']
[2024-04-05 16:23:30,590::test::INFO] [ARGS::save_dir] './results'
[2024-04-05 16:23:30,590::test::INFO] [ARGS::device] 'cuda'
[2024-04-05 16:23:30,590::test::INFO] [ARGS::dataset_path] './data/shapenet.hdf5'
[2024-04-05 16:23:30,590::test::INFO] [ARGS::batch_size] 128
[2024-04-05 16:23:30,771::test::INFO] Loading datasets...
[2024-04-05 16:23:31,838::test::INFO] Loading model...
100% 41/41 [03:55<00:00,  5.73s/it]
[2024-04-05 16:27:27,120::test::INFO] Saving point clouds...
[2024-04-05 16:27:32,617::test::INFO] Start computing metrics...
EMD-CD:   0% 0/41 [00:00<?, ?it/s]

[WARNING]
  * EMD is not implemented due to GPU 

In [ ]:
!python3 test_ae.py --ckpt "./pretrained/AE_augmented_chair.pt" --categories chair


[2024-04-10 13:23:55,725::test::INFO] [ARGS::ckpt] './pretrained/AE_augmented_chair.pt'
[2024-04-10 13:23:55,726::test::INFO] [ARGS::categories] ['chair']
[2024-04-10 13:23:55,727::test::INFO] [ARGS::save_dir] './results'
[2024-04-10 13:23:55,728::test::INFO] [ARGS::device] 'cuda'
[2024-04-10 13:23:55,728::test::INFO] [ARGS::dataset_path] './data/shapenet.hdf5'
[2024-04-10 13:23:55,728::test::INFO] [ARGS::batch_size] 128
[2024-04-10 13:23:55,920::test::INFO] Loading datasets...
[2024-04-10 13:23:56,116::test::INFO] Loading model...
100% 8/8 [00:44<00:00,  5.54s/it]
[2024-04-10 13:24:40,486::test::INFO] Saving point clouds...
[2024-04-10 13:24:40,582::test::INFO] Start computing metrics...
EMD-CD:   0% 0/8 [00:00<?, ?it/s]

[WARNING]
  * EMD is not implemented due to GPU compatability issue.
  * We will set all EMD to zero by default.
  * You may implement your own EMD in the function `emd_approx` in ./evaluation/evaluation_metrics.py


EMD-CD: 100% 8/8 [00:00<00:00, 35.30it/s]
[2024-04

In [ ]:
# test using original model
# !python3 test_ae.py --ckpt ./pretrained/AE_all.pt --categories "chair,table,car,sofa,pot,microwave,bathtub,can,speaker,lamp,printer,file,bed,cabinet,vessel,bus,washer,dishwasher,clock,bowl,tin_can,jar,camera"



[2024-04-05 16:28:26,279::test::INFO] [ARGS::ckpt] './pretrained/AE_all.pt'
[2024-04-05 16:28:26,280::test::INFO] [ARGS::categories] ['chair', 'table', 'car', 'sofa', 'pot', 'microwave', 'bathtub', 'can', 'speaker', 'lamp', 'printer', 'file', 'bed', 'cabinet', 'vessel', 'bus', 'washer', 'dishwasher', 'clock', 'bowl', 'tin_can', 'jar', 'camera']
[2024-04-05 16:28:26,282::test::INFO] [ARGS::save_dir] './results'
[2024-04-05 16:28:26,283::test::INFO] [ARGS::device] 'cuda'
[2024-04-05 16:28:26,283::test::INFO] [ARGS::dataset_path] './data/shapenet.hdf5'
[2024-04-05 16:28:26,283::test::INFO] [ARGS::batch_size] 128
[2024-04-05 16:28:26,447::test::INFO] Loading datasets...
[2024-04-05 16:28:27,556::test::INFO] Loading model...
100% 41/41 [03:55<00:00,  5.75s/it]
[2024-04-05 16:32:23,704::test::INFO] Saving point clouds...
[2024-04-05 16:32:25,183::test::INFO] Start computing metrics...
EMD-CD:   0% 0/41 [00:00<?, ?it/s]

[WARNING]
  * EMD is not implemented due to GPU compatability issue.
  *

In [ ]:
# test using original model
!python3 test_ae.py --ckpt ./pretrained/AE_all.pt --categories chair


[2024-04-10 13:24:43,357::test::INFO] [ARGS::ckpt] './pretrained/AE_all.pt'
[2024-04-10 13:24:43,359::test::INFO] [ARGS::categories] ['chair']
[2024-04-10 13:24:43,359::test::INFO] [ARGS::save_dir] './results'
[2024-04-10 13:24:43,359::test::INFO] [ARGS::device] 'cuda'
[2024-04-10 13:24:43,359::test::INFO] [ARGS::dataset_path] './data/shapenet.hdf5'
[2024-04-10 13:24:43,359::test::INFO] [ARGS::batch_size] 128
[2024-04-10 13:24:43,508::test::INFO] Loading datasets...
[2024-04-10 13:24:43,705::test::INFO] Loading model...
100% 8/8 [00:44<00:00,  5.55s/it]
[2024-04-10 13:25:28,167::test::INFO] Saving point clouds...
[2024-04-10 13:25:28,290::test::INFO] Start computing metrics...
EMD-CD:   0% 0/8 [00:00<?, ?it/s]

[WARNING]
  * EMD is not implemented due to GPU compatability issue.
  * We will set all EMD to zero by default.
  * You may implement your own EMD in the function `emd_approx` in ./evaluation/evaluation_metrics.py


EMD-CD: 100% 8/8 [00:00<00:00, 34.05it/s]
[2024-04-10 13:25:28

In [ ]:
import torch
torch.cuda.empty_cache()
